In [1]:
import pandas as pd

In [26]:
df_upheno_all=pd.read_csv("upheno_associated_entities.tsv", sep="\t")
df_upheno_all.columns = ['phenotype','entity']
df_upheno_all = df_upheno_all.replace({'<http://purl.obolibrary.org/obo/': '', '>':'','_':':'}, regex=True)
df_upheno_all

,phenotype,entity
0,HP:0031406,GO:0019221
1,HP:0012347,GO:0006487
2,HP:0012359,GO:0036066
3,HP:0012358,GO:0006493
4,HP:0410359,CHEBI:546807
...,...,...
5374,MP:0001680,GO:0007498
5375,MP:0001675,GO:0007398
5376,MP:0005221,GO:0009948
5377,MP:0001705,GO:0009946


In [8]:
df_map=pd.read_csv("map.tsv", sep="\t", header=None)
df_map.columns = ['term1','term2']
df_map

,term1,term2
0,AMNIOTIC_EPITHELIAL_CELL,ABNORMAL_OF_PLACENTAL_MEMBRANES
1,ASTROCYTE,ABNORMAL_ASTROCYTE_MORPHOLOGY
2,BASOPHIL,ABNORMAL_BASOPHIL_MORPHOLOGY
3,BRONCHIAL_SMOOTH_MUSCLE_CELL,ABNORMAL_BRONCHUS_MORPHOLOGY
4,CARDIAC_FIBROBLAST,ABNORMAL_HEART_MORPHOLOGY
...,...,...
87,TONSIL,ABNORMAL_TONSIL_MORPHOLOGY
88,UMBILICAL_CORD,ABNORMAL_UMBILICAL_CORD_MORPHOLOGY
89,URINARY_BLADDER,ABNORMAL_BLADDER_MORPHOLOGY
90,UTERUS,ABNORMAL_UTERUS_MORPHOLOGY


In [9]:
df_terms=pd.read_csv("terms.tsv", sep="\t", header=None)
df_terms.columns = ['term_name','term_id']
df_terms

,term_name,term_id
0,ACINAR_CELL,CL:0000622
1,AMNIOTIC_EPITHELIAL_CELL,CL:0002536
2,ABNORMAL_OF_PLACENTAL_MEMBRANES,HP:0011409
3,ASTROCYTE,CL:0000127
4,BASOPHIL,CL:0000767
...,...,...
182,URINARY_BLADDER,UBERON:0001255
183,ABNORMAL_BLADDER_MORPHOLOGY,HP:0025487
184,UTERUS,UBERON:0000995
185,VAGINA,UBERON:0000996


In [37]:
print(len(df_map))
df = pd.merge(df_map, df_terms, how='left', left_on=['term1'], right_on=['term_name'])
print(len(df))
df = pd.merge(df, df_terms, how='left', left_on=['term2'], right_on=['term_name'])
df.drop_duplicates(inplace=True)
print(len(df))
df=df[['term_id_x','term_id_y']]
df.columns = ['object_id','subject_id']
df['predicate_id'] = "RO:0000052"
df.to_csv("phenol_uberon_map_sssom.tsv",sep="\t",index=False)
df

92
92
92


,object_id,subject_id,predicate_id
0,CL:0002536,HP:0011409,RO:0000052
1,CL:0000127,HP:0100707,RO:0000052
2,CL:0000767,HP:0001912,RO:0000052
3,CL:0002598,HP:0025426,RO:0000052
4,CL:0002548,HP:0001627,RO:0000052
...,...,...,...
95,UBERON:0002372,HP:0100765,RO:0000052
96,UBERON:0002331,HP:0010881,RO:0000052
97,UBERON:0001255,HP:0025487,RO:0000052
98,UBERON:0000995,HP:0031105,RO:0000052


In [38]:
import numpy as np
df_upheno=pd.merge(df, df_upheno_all, how='left', left_on=['subject_id'], right_on=['phenotype'])
del df_upheno['phenotype']
df_upheno['cat'] = np.where(df_upheno['entity'].isna(),"NOMAP",np.where(df_upheno['object_id']==df_upheno['entity'],"SAME","DIFFERENT"))
df_upheno

,object_id,subject_id,predicate_id,entity,cat
0,CL:0002536,HP:0011409,RO:0000052,UBERON:0009002,DIFFERENT
1,CL:0000127,HP:0100707,RO:0000052,CL:0000127,SAME
2,CL:0000767,HP:0001912,RO:0000052,CL:0000767,SAME
3,CL:0002598,HP:0025426,RO:0000052,UBERON:0002185,DIFFERENT
4,CL:0002548,HP:0001627,RO:0000052,UBERON:0000948,DIFFERENT
...,...,...,...,...,...
87,UBERON:0002372,HP:0100765,RO:0000052,UBERON:0002372,SAME
88,UBERON:0002331,HP:0010881,RO:0000052,UBERON:0002331,SAME
89,UBERON:0001255,HP:0025487,RO:0000052,NaN,NOMAP
90,UBERON:0000995,HP:0031105,RO:0000052,NaN,NOMAP


In [39]:
df_upheno['cat'].value_counts()

DIFFERENT    39
SAME         31
NOMAP        22
Name: cat, dtype: int64

In [40]:
df_upheno.to_csv("comp_upheno.tsv",sep="\t",index=False)